In [19]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import os
import glob
import cv2
from sklearn.model_selection import train_test_split
import shutil
import matplotlib.pyplot as plt


In [20]:
# Load the MRI brain images and labels
images = []
labels = []
for filename in glob.glob(r'F:\Project1\images\New Dataset\Detection\CLAHE-RMSHE\Tumor\*.jpg'):
    img = plt.imread(filename)
    images.append(img)
    labels.append(1)

for filename in glob.glob(r'F:\Project1\images\New Dataset\Detection\CLAHE-RMSHE\No Tumor\*.jpg'):
    img = plt.imread(filename)
    images.append(img)
    labels.append(0)

# # Split the data into training, validation, and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# # Save the split data in separate directories
for i, image in enumerate(train_images):
    path1 = r"F:\Project1\images\New Dataset\Detection\CLAHE-RMSHE\Train\Tumor/"+str(i)+".jpg"
    path2 = r"F:\Project1\images\New Dataset\Detection\CLAHE-RMSHE\Train\No Tumor/"+str(i)+".jpg"
    if train_labels[i] == 1:
        cv2.imwrite(path1, image)
    else:
        cv2.imwrite(path2, image)
        
for i, image in enumerate(test_images):
    path1 = r"F:\Project1\images\New Dataset\Detection\CLAHE-RMSHE\Test\Tumor/"+str(i)+".jpg"
    path2 = r"F:\Project1\images\New Dataset\Detection\CLAHE-RMSHE\Test\No Tumor/"+str(i)+".jpg"
    if test_labels[i] == 1:
        cv2.imwrite(path1, image)
    else:
        cv2.imwrite(path2, image)
        
for i, image in enumerate(val_images):
    path1 = r"F:\Project1\images\New Dataset\Detection\CLAHE-RMSHE\Validation\Tumor/"+str(i)+".jpg"
    path2 = r"F:\Project1\images\New Dataset\Detection\CLAHE-RMSHE\Validation\No Tumor/"+str(i)+".jpg"
    if val_labels[i] == 1:
        cv2.imwrite(path1, image)
    else:
        cv2.imwrite(path2, image)

In [21]:
#CNN Model

model=Sequential()

model.add(Conv2D(filters=16, kernel_size= (3,3), activation= 'relu', input_shape = (224,224,3)))

model.add(Conv2D(filters=36, kernel_size= (3,3), activation= 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size= (3,3), activation= 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size= (3,3), activation= 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 16)      448       
                                                                 
 conv2d_5 (Conv2D)           (None, 220, 220, 36)      5220      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 110, 110, 36)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 108, 108, 64)      20800     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 52, 52, 128)      

In [22]:
model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

In [23]:
def preprocessingImages1(path):
    """
    input : Path
    output : Pre processed images
    """
    image_data = ImageDataGenerator(zoom_range=0.2, shear_range=0.2, rescale=1/255, horizontal_flip=True)
    image = image_data.flow_from_directory(directory = path, target_size = (224,224), batch_size = 32, class_mode = 'binary')
    
    return image

In [24]:
#put training data location in path
path = r"F:\Project1\images\New Dataset\Detection\CLAHE-RMSHE\Train"
train_data = preprocessingImages1(path)

Found 128 images belonging to 2 classes.


In [25]:
def preprocessingImages2(path):
    """
    input : Path
    output : Pre processed images
    """
    image_data = ImageDataGenerator(rescale=1/255)
    image = image_data.flow_from_directory(directory = path, target_size = (224,224), batch_size = 32, class_mode = 'binary')
    
    return image

In [26]:
#put test data location in path
path = r"F:\Project1\images\New Dataset\Detection\CLAHE-RMSHE\Test"
test_data = preprocessingImages2(path)

Found 40 images belonging to 2 classes.


In [27]:
#put validation data location in path
path = r"F:\Project1\images\New Dataset\Detection\CLAHE-RMSHE\Validation"
val_data = preprocessingImages2(path)

Found 32 images belonging to 2 classes.


In [28]:
#Early stopping and model check point
from keras.callbacks import ModelCheckpoint, EarlyStopping

#early stopping
es = EarlyStopping(monitor="val_loss", min_delta=0.01, patience=6, verbose=1, mode="auto")

#model check point
#save the best model as "./bestmodel.h5" and give path
mc = ModelCheckpoint(monitor="val_accuracy", filepath="./bestmodel.h5", verbose=1, save_best_only=True, mode='auto')

cd = [es,mc]

In [29]:
#training
hs = model.fit_generator(generator=train_data, steps_per_epoch=2, epochs=4, verbose=1, validation_data=val_data, validation_steps=16, callbacks=cd)

C:\Users\Rakesh\AppData\Local\Temp\ipykernel_29184\789910006.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hs = model.fit_generator(generator=train_data, steps_per_epoch=2, epochs=4, verbose=1, validation_data=val_data, validation_steps=16, callbacks=cd)


Epoch 1/4
2/2 [==============================] - ETA: 0s - loss: 0.6440 - accuracy: 0.5000WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 16 batches). You may need to use the repeat() function when building your dataset.

Epoch 1: val_accuracy improved from -inf to 0.81250, saving model to .\bestmodel.h5
2/2 [==============================] - 10s 4s/step - loss: 0.6440 - accuracy: 0.5000 - val_loss: 0.4829 - val_accuracy: 0.8125
Epoch 2/4
2/2 [==============================] - 6s 3s/step - loss: 0.7315 - accuracy: 0.6250
Epoch 3/4
2/2 [==============================] - 6s 3s/step - loss: 0.6131 - accuracy: 0.5938
Epoch 4/4
2/2 [==============================] - 6s 3s/step - loss: 0.6324 - accuracy: 0.7188


In [30]:
h = hs.history
h.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [31]:
from keras.models import load_model

model = load_model("./bestmodel.h5")

In [32]:
acc = model.evaluate(test_data)[1]
print(f"accuracy of our model is {acc}")

2/2 [==============================] - 1s 155ms/step - loss: 0.7630 - accuracy: 0.6000
accuracy of our model is 0.6000000238418579


In [15]:
train_data.class_indices

{'No Tumor': 0, 'Tumor': 1}

In [16]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
path = "./images/TEMP/brain_tumor_dataset/no/50 no.jpg"
img = load_img(path, target_size=(224, 224))
input_arr = img_to_array(img)/255
input_arr.shape

input_arr  = np.expand_dims(input_arr, axis=0)
pred = (model.predict(input_arr) > 0.5).astype("int32")
if pred[0][0]==0:
    print("No Tumor Detected")
else:
    print("Tumor Detected")

1/1 [==============================] - 0s 481ms/step
No Tumor Detected


In [ ]:
h = hs.history
h.keys()
from keras.models import load_model
model = load_model("./bestmodel.h5")
acc = model.evaluate(test_data)[1]
print(f"accuracy of our model is {acc}")